In [2]:
import tensorflow as tf
import numpy as np
import scipy as sp
import scipy.stats as stats
import multiprocessing as mp
import itertools

In [3]:
N = stats.norm.cdf

def BlackScholesCall(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * N(d1) - K * np.exp(-r*T)* N(d2)

In [6]:
S = tf.constant(100.0)

with tf.GradientTape() as g:
    g.watch(S)
    C = BlackScholesCall(S,100,1,0,0.2)
    delta = g.gradient(C,S)

In [8]:
print(delta)

tf.Tensor(0.5398278, shape=(), dtype=float32)
